# TSA Chapter 4: SARIMA Model Diagnostics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch4/TSA_ch4_diagnostics/TSA_ch4_diagnostics.ipynb)

Residual diagnostics for SARIMA(0,1,1)(0,1,1)_12 airline model: time series plot, histogram, ACF, and Q-Q plot.

In [ ]:
!pip install numpy pandas matplotlib statsmodels scipy -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import acf, pacf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Load airline passengers
data = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv',
                    parse_dates=['Month'], index_col='Month')
passengers = data['Passengers']
passengers.index.freq = 'MS'
log_passengers = np.log(passengers)

In [ ]:
# Fit SARIMA(0,1,1)(0,1,1)_12 -- the airline model
best_model = SARIMAX(log_passengers, order=(0, 1, 1), seasonal_order=(0, 1, 1, 12),
                     enforce_stationarity=False, enforce_invertibility=False)
best_fit = best_model.fit(disp=False)
resid = best_fit.resid[13:]  # skip burn-in

fig, axes = plt.subplots(2, 2, figsize=(12, 7))

# Residuals over time
axes[0, 0].plot(resid.index, resid.values, color=BLUE, linewidth=0.8)
axes[0, 0].axhline(y=0, color=GRAY, linestyle='--', alpha=0.5)
axes[0, 0].set_title('Residuals over Time', fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Residual')

# Residual histogram
resid_clean = resid.dropna()
axes[0, 1].hist(resid_clean, bins=25, density=True, color=BLUE, alpha=0.7, edgecolor='white')
x_range = np.linspace(resid_clean.min(), resid_clean.max(), 100)
axes[0, 1].plot(x_range,
                stats.norm.pdf(x_range, resid_clean.mean(), resid_clean.std()),
                color=RED, linewidth=2, label='Normal Fit')
axes[0, 1].set_title('Residual Distribution', fontweight='bold')
axes[0, 1].set_xlabel('Residual')
axes[0, 1].set_ylabel('Density')
axes[0, 1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), frameon=False)

# ACF of residuals
acf_resid = acf(resid_clean, nlags=36)
ci_r = 1.96 / np.sqrt(len(resid_clean))
axes[1, 0].bar(range(len(acf_resid)), acf_resid, color=BLUE, width=0.35)
axes[1, 0].axhline(y=0, color='black', linewidth=0.5)
axes[1, 0].axhline(y=ci_r, color=RED, linestyle='--', alpha=0.7)
axes[1, 0].axhline(y=-ci_r, color=RED, linestyle='--', alpha=0.7)
for lag in [12, 24, 36]:
    axes[1, 0].axvline(x=lag, color=ORANGE, linestyle=':', alpha=0.3)
axes[1, 0].set_title('ACF of Residuals', fontweight='bold')
axes[1, 0].set_xlabel('Lag')
axes[1, 0].set_ylabel('ACF')

# Q-Q plot
std_resid = (resid_clean - resid_clean.mean()) / resid_clean.std()
(osm, osr), _ = stats.probplot(std_resid, dist='norm', fit=True)
axes[1, 1].scatter(osm, osr, color=BLUE, alpha=0.6, s=30, edgecolors='none')
axes[1, 1].plot([-3, 3], [-3, 3], color=RED, linewidth=2, linestyle='--', label='45-degree line')
axes[1, 1].set_xlim(-3.5, 3.5)
axes[1, 1].set_ylim(-3.5, 3.5)
axes[1, 1].set_title('Q-Q Plot (Normal)', fontweight='bold')
axes[1, 1].set_xlabel('Theoretical Quantiles')
axes[1, 1].set_ylabel('Sample Quantiles')
axes[1, 1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), frameon=False)

plt.tight_layout(); save_chart(fig, 'ch4_diagnostics'); plt.show()